In [ ]:
%spark.pyspark

market_events = spark.table("market.events")
# market_events = spark.table("kruglov.market_events")
# market_events = spark.read.parquet("/apps/hive/warehouse/eakotelnikov.db/market_events")

market_events.show(5)
market_events.printSchema()

Вывести топ категорий по количеству просмотров товаров за всё время

In [1]:
%pyspark

# вывести в виде
# +--------------------------------+----------+
# |category_code                   |view_count|
# +--------------------------------+----------+
# |null                            |20837460  |
# |electronics.smartphone          |14832387  |
# |computers.notebook              |2103024   |

from pyspark.sql.functions import count, col

market_events.filter(col("event_type") == "view") \
             .groupBy("category_code") \
             .agg(count("*").alias("view_count")) \
             .orderBy(col("view_count").desc()) \
             .show()

Построить гистограмму распределения цен на проданные товары за 10 октября

In [2]:
%pyspark

# результат визуализировать через z.show()

from pyspark.sql.functions import count, col

hist_data = market_events.filter((col("event_date") == "2019-10-10") & (col("event_type") == "purchase")) \
                         .groupBy("price") \
                         .agg(count("*").alias("count")) \
                         .orderBy(col("price").asc())
                         
hist_data.show()
z.show(hist_data)

In [3]:
%pyspark
# Решение с бинами через бакетирование
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.ml.feature import Bucketizer

# Фильтруем датафрейм
filtered_data = market_events.filter((F.col("event_date") == "2019-10-10") & (F.col("event_type") == "purchase"))

# Определяем границы диапазонов
bins = 20
max_price = filtered_data.agg({"price": "max"}).collect()[0][0]
step = max_price / bins
boundaries = [i * step for i in range(bins+1)]

# Создаем Bucketizer
bucketizer = Bucketizer(splits=boundaries, inputCol="price", outputCol="price_bucket")

# Применяем Bucketizer к датафрейму
filtered_data = bucketizer.transform(filtered_data)

hist_data = filtered_data.select("price", "price_bucket") \
                          .groupBy("price_bucket") \
                          .agg(F.count("*").alias("bucket_count")) \
                          .orderBy(F.col("price_bucket").asc())
            

# Выводим результат
print(f"Шаг бакета {step} руб.") # шаг каждого бакета
hist_data.show()
z.show(hist_data)

Посчитать количество продаж за октябрь отдельно бренда apple и остальных

In [4]:
%pyspark

# вывести через df.show() в виде
# +--------+------+
# |is_apple| count|
# +--------+------+
# |    true| *****|
# |   false| *****|
# +--------+------+

from pyspark.sql import functions as F

apple_oct_data = market_events.filter((F.month("event_date") == 10) & (col("event_type") == "purchase")) \
                              .na.fill({"brand": "No_brand"}) \
                              .groupBy("brand") \
                              .agg(count("*").alias("brand_count")) \
                              .withColumn("is_apple", col("brand") == "apple") \
                              .groupBy("is_apple") \
                              .agg(F.sum("brand_count").alias("count"))

apple_oct_data.show()

Построить почасовой график продаж и прибыли за вторую неделю октября

In [5]:
%pyspark

# z.show(), ключ -- часы, значения -- число продаж и сумма прибыли за этот час

from pyspark.sql import functions as F

data_by_hour = market_events.filter((F.weekofyear("event_date") == 41) & (F.col("event_type") == "purchase")) \
                .withColumn("hour", F.hour("event_time")) \
                .groupBy("hour") \
                .agg(F.count("price").alias("sales_count"), F.sum("price").alias("hour_revenue"))

data_by_hour.show()
z.show(data_by_hour)

Построить почасовой график продаж и прибыли за вторую неделю октября

In [6]:
%pyspark

# z.show(), ключ -- час в диапазоне от 0 до 23, значение -- усредненное за месяц число продаж в этот час на месячных данных

from pyspark.sql import functions as F

avg_data_by_hour = market_events.filter((F.month("event_date") == 10) & (F.col("event_type") == "purchase")) \
                        .withColumn("hour", F.hour("event_time")) \
                        .withColumn("day", F.dayofmonth("event_date")) \
                        .groupBy(["hour", "day"]) \
                        .agg(F.count("price").alias("sales_count"), F.sum("price").alias("hour_revenue")) \
                        .groupBy(["hour"]) \
                        .agg(F.avg("sales_count").alias("avg_sales_count"), F.avg("hour_revenue").alias("avg_hour_revenue"))
                        
avg_data_by_hour.show()
z.show(avg_data_by_hour)

In [7]:
%pyspark